In [2]:
import IPython
import numpy as np
import networkx as nx
import requests
import pandas as pd
import csv
import matplotlib.pyplot as plt
import time
from fonctions import *
pd.set_option('mode.chained_assignment', None)

In [25]:
### Importing the nodes as a DataFrame for easier manipulations ###

dataFrame = dataFrame = pd.read_csv('Results/nodes.csv', encoding='utf-8')
dataFrame['Label'] = dataFrame['Label'].astype('string')
print(dataFrame.info())
display(dataFrame)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4604 entries, 0 to 4603
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                4604 non-null   int64  
 1   Label             4604 non-null   string 
 2   modularity_class  4604 non-null   int64  
 3   Cluster           4604 non-null   int64  
 4   Degree            4604 non-null   int64  
 5   clustering        4604 non-null   float64
dtypes: float64(1), int64(4), string(1)
memory usage: 215.9 KB
None


,Id,Label,modularity_class,Cluster,Degree,clustering
0,55232472,2019_CFL_season,19,0,15,0.285714
1,3946242,Eimuntas_Nekro?ius,6,5,1,0.000000
2,8161118,Bobby_Carpenter_(American_football),19,0,3,0.333333
3,51544610,Florence_Kasumba,2,4,5,1.000000
4,692976,Beorhtwulf_of_Mercia,27,14,13,0.910256
...,...,...,...,...,...,...
4599,36280977,List_of_Mayflower_passengers_who_died_at_sea_N...,6,5,5,1.000000
4600,46646353,Lucy_Frazer,1,2,2,0.000000
4601,48738235,List_of_highest_historical_scores_in_figure_sk...,15,3,1,0.000000
4602,653621,Michael_Johnson_(sprinter),12,1,2,0.000000


In [26]:
### Extract nodes labels from Wikipedia queries ###



# URL for the quieries
urls = "https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&format=json&pageids="


i=0
Nb_pages = len(dataFrame)


### Associate the 'Qid' value of Wikipedia pages in the DataFrame ###
### Give '-1' value if an error has occured during the query      ###

for pageId in np.int64(dataFrame['Id']):
    response = requests.get(urls + str(pageId)).json()
    try:
        Qid = (list(findkeys(response, 'wikibase_item'))[0])
        Title = (list(findkeys(response, 'title'))[0])
    except IndexError:
        dataFrame.loc[dataFrame['Id'] == pageId, 'Qid'] = '-1'
        i+=1
        print(i,"/", Nb_pages, dataFrame.loc[dataFrame['Id'] == pageId, 'Label'], "\t error")
    else:
        dataFrame.loc[dataFrame['Id'] == pageId, 'Qid'] = Qid
        dataFrame.loc[dataFrame['Id'] == pageId, 'Label'] = Title
        i+=1
        print(i,"/", Nb_pages, dataFrame.loc[dataFrame['Id'] == pageId, 'Label'].values[0])

# Clear the loading display
IPython.display.clear_output(wait=False)

        
# Save the DataFrame with their associated Qids
dataFrame.to_csv('Results/pages_Qids.csv', encoding='utf-8')

dataFrame

,Id,Label,modularity_class,Cluster,Degree,clustering,Qid
0,55232472,2019 CFL season,19,0,15,0.285714,Q60526029
1,3946242,Eimuntas Nekrošius,6,5,1,0.000000,Q2525451
2,8161118,Bobby Carpenter (American football),19,0,3,0.333333,Q4845743
3,51544610,Florence Kasumba,2,4,5,1.000000,Q1429374
4,692976,Beorhtwulf of Mercia,27,14,13,0.910256,Q818588
...,...,...,...,...,...,...,...
4599,36280977,List of Mayflower passengers who died at sea N...,6,5,5,1.000000,Q6797245
4600,46646353,Lucy Frazer,1,2,2,0.000000,Q19957811
4601,48738235,List of highest historical scores in figure sk...,15,3,1,0.000000,Q25036770
4602,653621,Michael Johnson (sprinter),12,1,2,0.000000,Q190924


In [8]:
### Extracting topic pages from database API ###


dataFrame = pd.read_csv('Results/pages_Qids.csv')

# Match_topic_API = pd.DataFrame(columns = ['Qid', 'topic', 'probability', 'page_id', 'page_title', 'modularity_class', 'wiki_db'])
Match_topic_API = pd.DataFrame()


                
### The API's URL from the topic is extracted ###         
urls = "http://86.119.25.229:5000/api/v1/wikidata/topic?qid="
threshold = '0.1'

    
tmps1=time.time()
    
a=0 
n = dataFrame.index[-1]
for pageIndex in dataFrame.index:
    pageQid = dataFrame.at[pageIndex, 'Qid']
    pageModularity = dataFrame.at[pageIndex, 'modularity_class']
    response = requests.get(urls + pageQid + "&threshold=" + threshold).json()
    scores = list(findkeys(response, 'score'))
    topics = list(findkeys(response, 'topic'))
    
    try:
        page_title = response['name']
    except KeyError:
        page_title = dataFrame.at[pageIndex, 'Label']
    for i in range(len(scores)):
        page = dataFrame.iloc[pageIndex]
        page['Topic'] = topics[i]
        page['Probability'] = scores[i]
        Match_topic_API = Match_topic_API.append(page, ignore_index=True)
    print(a, "/",n)
    a+=1
    
    
# Clear the loading display
IPython.display.clear_output(wait=False)
tmps2=time.time()
print("Execution time = ",(tmps2-tmps1), " s")

# Save the results 
Match_topic_API.drop(columns = ['Unnamed: 0'], inplace = True )
Match_topic_API.to_csv('Results/Match_topic_API.csv', encoding='utf-8')

Match_topic_API

Execution time =  2080.686254262924  s


,Cluster,Degree,Id,Label,Probability,Qid,Topic,clustering,modularity_class
0,0.0,15.0,55232472.0,2019 CFL season,0.724880,Q60526029,Culture.Sports,0.285714,19.0
1,0.0,15.0,55232472.0,2019 CFL season,0.000000,Q60526029,Geography.Regions.Europe.Europe*,0.285714,19.0
2,0.0,15.0,55232472.0,2019 CFL season,0.000000,Q60526029,Geography.Regions.Oceania,0.285714,19.0
3,0.0,15.0,55232472.0,2019 CFL season,0.000000,Q60526029,Geography.Regions.Europe.Northern_Europe,0.285714,19.0
4,5.0,1.0,3946242.0,Eimuntas Nekrošius,0.992194,Q2525451,Culture.Biography.Biography*,0.000000,6.0
...,...,...,...,...,...,...,...,...,...
18773,1.0,2.0,653621.0,Michael Johnson (sprinter),0.000000,Q190924,Geography.Regions.Americas.North_America,0.000000,12.0
18774,16.0,6.0,346783.0,List of television stations in Alaska,1.000000,Q6642662,Compilation.List_Disambig,0.533333,8.0
18775,16.0,6.0,346783.0,List of television stations in Alaska,0.996737,Q6642662,Culture.Media.Television,0.533333,8.0
18776,16.0,6.0,346783.0,List of television stations in Alaska,0.995811,Q6642662,Culture.Media.Media*,0.533333,8.0


In [4]:



def string_separator(df):
    for index, row in df.iterrows():
        topic_str = row['Topic']
        topic_list = topic_str.split('.') 
        df.loc[df['Topic'] == topic_str, 'Subtopic'] = topic_list[-1]

        

dataFrame = pd.read_csv('Results/Match_topic_API.csv', index_col = 'Unnamed: 0')





# Creating and returning new DataFrame 'modularity_df' for represetating clusters topics 
def find_main_topics_clusters(cluster_name, dataFrame):
    
    
    modularity_df = pd.DataFrame({'Main_Topics':[], 'Higher_Degree_Article':[]})
    
    # Deleting 'Biography' because it's overrepresented and not 
    dataFrame_without_biography = dataFrame.loc[dataFrame['Topic'] != 'Culture.Biography.Biography*']

    topic_group = ((dataFrame_without_biography.groupby(['Topic', cluster_name]))).count()

   
    total_cluster = ((dataFrame_without_biography.groupby([cluster_name]))).count()
    total_cluster = total_cluster['Qid'].to_frame()
    total_cluster.rename(columns={'Qid':'Total'}, inplace=True)

    topic_group = topic_group['Qid'].to_frame().reset_index('Topic')
    topic_group.rename(columns={'Qid':'Count'}, inplace=True)
    topic_group.sort_values(by = [cluster_name, 'Count'], inplace=True, ascending = [True, False])

    topic_group['Total'] = total_cluster['Total']
    topic_group['Ratio'] = topic_group['Count'] / topic_group['Total']
    topic_group['Subtopic'] = ''
    string_separator(topic_group)



    nb_topic = int( dataFrame[cluster_name].max() ) + 1

    for i in range(nb_topic):
        str_main_topics = topic_group.loc[i, 'Subtopic'].values[0] + ' (' + str((topic_group.loc[i, 'Ratio'].values[0]).round(decimals=2)) + ') '
        str_main_topics += '/ ' + topic_group.loc[i, 'Subtopic'].values[1] + ' (' + str((topic_group.loc[i, 'Ratio'].values[1]).round(decimals=2)) + ') '
        str_main_topics += '/ ' + topic_group.loc[i, 'Subtopic'].values[2] + ' (' + str((topic_group.loc[i, 'Ratio'].values[2]).round(decimals=2)) + ') '
        dataFrame.loc[dataFrame[cluster_name] == i, (cluster_name+'_Topic')] = str_main_topics
        
        page_id_max_degree = dataFrame.loc[dataFrame[cluster_name] == i, 'Degree'].idxmax()
        str_degree = dataFrame.loc[page_id_max_degree, 'Label'] + ' (' + str(int(dataFrame.loc[page_id_max_degree, 'Degree'])) + ')'
        
        mod = {'Main_Topics':str_main_topics, 'Higher_Degree_Article':str_degree}
        modularity_df = modularity_df.append(mod, ignore_index=True)
        
    return modularity_df
cluster_name = 'modularity_class'   



modularity_df = find_main_topics_clusters(cluster_name, dataFrame) 


modularity_df.index.rename('modularity_class', inplace = True)

modularity_df.to_csv('Results/topic_by_modularity.csv', encoding='utf-8')

display(dataFrame)

modularity_df



,Cluster,Degree,Id,Label,Probability,Qid,Topic,clustering,modularity_class,modularity_class_Topic
0,0.0,15.0,55232472.0,2019 CFL season,0.724880,Q60526029,Culture.Sports,0.285714,19.0,North_America (0.33) / Sports (0.32) / Media* ...
1,0.0,15.0,55232472.0,2019 CFL season,0.000000,Q60526029,Geography.Regions.Europe.Europe*,0.285714,19.0,North_America (0.33) / Sports (0.32) / Media* ...
2,0.0,15.0,55232472.0,2019 CFL season,0.000000,Q60526029,Geography.Regions.Oceania,0.285714,19.0,North_America (0.33) / Sports (0.32) / Media* ...
3,0.0,15.0,55232472.0,2019 CFL season,0.000000,Q60526029,Geography.Regions.Europe.Northern_Europe,0.285714,19.0,North_America (0.33) / Sports (0.32) / Media* ...
4,5.0,1.0,3946242.0,Eimuntas Nekrošius,0.992194,Q2525451,Culture.Biography.Biography*,0.000000,6.0,Media* (0.19) / North_America (0.16) / Films (...
...,...,...,...,...,...,...,...,...,...,...
18773,1.0,2.0,653621.0,Michael Johnson (sprinter),0.000000,Q190924,Geography.Regions.Americas.North_America,0.000000,12.0,Sports (0.36) / Europe* (0.2) / Northern_Europ...
18774,16.0,6.0,346783.0,List of television stations in Alaska,1.000000,Q6642662,Compilation.List_Disambig,0.533333,8.0,North_America (0.2) / STEM* (0.1) / Politics_a...
18775,16.0,6.0,346783.0,List of television stations in Alaska,0.996737,Q6642662,Culture.Media.Television,0.533333,8.0,North_America (0.2) / STEM* (0.1) / Politics_a...
18776,16.0,6.0,346783.0,List of television stations in Alaska,0.995811,Q6642662,Culture.Media.Media*,0.533333,8.0,North_America (0.2) / STEM* (0.1) / Politics_a...


,Main_Topics,Higher_Degree_Article
modularity_class,,
0,Sports (0.35) / Europe* (0.25) / Northern_Euro...,Michael Parkhurst (14)
1,Europe* (0.15) / Northern_Europe (0.13) / Medi...,2017 United Kingdom general election (21)
2,Media* (0.2) / North_America (0.1) / Europe* (...,The Lion King (42)
3,North_America (0.22) / Politics_and_government...,2016 United States presidential election (48)
4,Media* (0.18) / North_America (0.16) / Films (...,The X-Files (20)
5,Asia* (0.14) / Europe* (0.08) / South_Asia (0....,Andaman Islands (37)
6,Media* (0.19) / North_America (0.16) / Films (...,Ambareesh (32)
7,STEM* (0.19) / Space (0.1) / Asia* (0.06),Arecibo message (16)
8,North_America (0.2) / STEM* (0.1) / Politics_a...,"Seward, Alaska (8)"
